In [8]:
import pandas as pd
import numpy as np
import RNG
from scipy.optimize import curve_fit

### Simulation parameters and helper functions

In [13]:
# num_order = np.random.randint(low=5,high=11,size=1)[0] # ~ Unif[1,11)
num_order = 8

class ActionLogger(object):
    def __init__(self):
        self.clock = 0.0
        self.n = 0
        self.alog = []
        self.tlog = []

    def write_act(self,a,t):
        self.clock += t
        self.n += 1
        self.alog.append(a)
        self.tlog.append(self.clock)

def exp_decay(x,a,b):
    return a*np.exp(-b*x)

def make_wheelset(logger,exp_means,rnd_seed):
    # sourcing
    for _ in range(7):
        logger.write_act("SpawnPart",RNG.Expon(exp_means[0],rnd_seed))

    # (rim + tire) x 2
    for _ in range(2):
        logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed)) # rim
        logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed)) # tire
        logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # rim + tire

    # (wheels + axle)
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed)) # axle
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # axle + wheel 1
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # axle + wheel 2
    
    # (axle + axle)
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed)) # connecting plate
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # axle + axle

    # inspection
    # logger.write_act("Inspection",RNG.Expon(insp_meant,rnd_seed))

def make_chassis_base(logger,exp_means,rnd_seed):
    # sourcing
    for _ in range(4):
        logger.write_act("SpawnPart",RNG.Expon(exp_means[0],rnd_seed))
    
    # (long plate + long plate)
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

    # (connecting plates) x 2
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

def make_front(logger,exp_means,rnd_seed):
    for _ in range(5):
        logger.write_act("SpawnPart",RNG.Expon(exp_means[0],rnd_seed))

    # (front bumper)
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # frist two blocks
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # last blocok

    # (engine bay)
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # slope 1
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed)) # slope 2 

def finish_station2(logger,exp_means,rnd_seed):
    # (wheelsets + base)
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))        

    # assemble base and front module
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

    # inspection
    # logger.write_act("Inspection",RNG.Expon(insp_meant,rnd_seed))

def make_steering(logger,exp_means,rnd_seed):
    for _ in range(2):
        logger.write_act("SpawnPart",RNG.Expon(exp_means[0],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

def finish_station3(logger,exp_means,rnd_seed):
    # windshield
    logger.write_act("SpawnPart",RNG.Expon(exp_means[0],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

    # (chassis + steering module)
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

    # inspection & pass to downstream
    # logger.write_act("Inspection",RNG.Expon(insp_meant,rnd_seed))

def make_body_frame(logger,exp_means,rnd_seed):
    for _ in range(7):
        logger.write_act("SpawnPart",RNG.Expon(exp_means[0],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

def finish_station4(logger,exp_means,rnd_seed):
    # ceiling
    logger.write_act("SpawnPart",RNG.Expon(exp_means[0],rnd_seed))
    logger.write_act("TeleportPart",RNG.Expon(exp_means[1],rnd_seed))
    logger.write_act("Assemble",RNG.Expon(exp_means[2],rnd_seed))

    # inspection & pass to downstream
    # logger.write_act("Inspection",RNG.Expon(insp_meant,rnd_seed))

def craft_car(logger,rnd_seed):
    make_wheelset(logger,rnd_seed)
    make_wheelset(logger,rnd_seed)
    make_chassis_base(logger,rnd_seed)
    make_front(logger,rnd_seed)
    finish_station2(logger,rnd_seed)
    make_steering(logger,rnd_seed)
    finish_station3(logger,rnd_seed)
    make_body_frame(logger,rnd_seed)
    finish_station4(logger,rnd_seed)

### Exponential decay of mean times of actions

In [14]:
# unit of time: minuate
spawn_meant0 = 0.5
teleport_meant0 = 0.5
assemble_meant0 = 1.0
pass_meant0 = 0.25
# insp_meant = 1.5

spawn_meant_param,_ = curve_fit(exp_decay,np.array([0,num_order-1]),np.array([spawn_meant0,spawn_meant0*0.5]))
teleport_meant_param,_ = curve_fit(exp_decay,np.array([0,num_order-1]),np.array([teleport_meant0,teleport_meant0*0.5]))
assemble_meant_param,_ = curve_fit(exp_decay,np.array([0,num_order-1]),np.array([assemble_meant0,assemble_meant0*0.5]))
pass_meant_param,_ = curve_fit(exp_decay,np.array([0,num_order-1]),np.array([pass_meant0,pass_meant0*0.5]))

## Collaboration - High
### Station 1 - Wheelsets x 2

In [15]:
x = np.arange(num_order*2)
spawn_meant = exp_decay(x,spawn_meant_param[0],spawn_meant_param[1])
teleport_meant = exp_decay(x,teleport_meant_param[0],teleport_meant_param[1])
assemble_meant = exp_decay(x,assemble_meant_param[0],assemble_meant_param[1])
pass_meant = exp_decay(x,pass_meant_param[0],pass_meant_param[1])
 
logger1 = ActionLogger()
for i in range(num_order):
    t = [spawn_meant[i],teleport_meant[i],assemble_meant[i]]
    make_wheelset(logger1,t,1)
    t = [spawn_meant[i+1],teleport_meant[i+1],assemble_meant[i+1]]
    make_wheelset(logger1,t,1)
    # pass wheelsets to downstream
    logger1.write_act("Pass",RNG.Expon(pass_meant[i],1))

In [24]:
np.arange(16)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

### Station 2 - Chassis

In [19]:
upstream_parts_idx = np.where(np.array(logger1.alog)=="Pass")[0]
upstream_parts_time = np.array(logger1.tlog)[upstream_parts_idx]

logger2 = ActionLogger()
num_received_up = 0
num_base = 0
num_complete = 0

# first chassis base
t = [spawn_meant[num_base],teleport_meant[num_base],assemble_meant[num_base]]
make_chassis_base(logger2,t,1)
num_base += 1

while True:
    # need to receive wheelsets from upstream
    if logger2.clock < upstream_parts_time[num_received_up]:
        # keep making bases until getting wheelsets from upstream
        t = [spawn_meant[num_base],teleport_meant[num_base],assemble_meant[num_base]]
        make_chassis_base(logger2,t,1)
        num_base += 1
    else:
        num_received_up += 1
        t = [spawn_meant[num_complete],teleport_meant[num_complete],assemble_meant[num_complete]]
        make_front(logger2,t,1)
        finish_station2(logger2,t,1)
        logger2.write_act("Pass",RNG.Expon(pass_meant[num_complete],1))
        num_complete += 1
    # completed all necessary chassis bases
    if num_base == num_order:
        break

# empty out chassis bases in inventory
while num_received_up < num_order:
    if logger2.clock < upstream_parts_time[num_received_up]:
        # wating wheelsets from upstream and complete chassis
        logger2.clock = upstream_parts_time[num_received_up]
    else:
        num_received_up += 1
        t = [spawn_meant[num_complete],teleport_meant[num_complete],assemble_meant[num_complete]]
        make_front(logger2,t,1)        
        finish_station2(logger2,t,1)
        logger2.write_act("Pass",RNG.Expon(pass_meant[num_complete],2))
        num_complete += 1

### Station 3 - Windshield & steering

In [20]:
upstream_parts_idx = np.where(np.array(logger2.alog)=="Pass")[0]
upstream_parts_time = np.array(logger2.tlog)[upstream_parts_idx]

logger3 = ActionLogger()
num_received_up = 0
num_steer = 0
num_complete = 0

# first steering
t = [spawn_meant[num_steer],teleport_meant[num_steer],assemble_meant[num_steer]]
make_steering(logger3,t,1)
num_steer += 1

while True:
    # need to receive chassis from upstream
    if logger3.clock < upstream_parts_time[num_received_up]:
        # keep making front body frame until getting chassis from upstream
        t = [spawn_meant[num_steer],teleport_meant[num_steer],assemble_meant[num_steer]]
        make_steering(logger3,t,1)
        num_steer += 1
    else:
        num_received_up += 1
        t = [spawn_meant[num_complete],teleport_meant[num_complete],assemble_meant[num_complete]]
        finish_station3(logger3,t,1)
        logger3.write_act("Pass",RNG.Expon(pass_meant[num_complete],3))
        num_complete += 1

    # completed all necessary front body frames
    if num_steer == num_order:
        break

# empty out steering modules in inventory
while num_received_up < num_order:
    if logger3.clock < upstream_parts_time[num_received_up]:
        # waiting chassis coming from upstream
        logger3.clock = upstream_parts_time[num_received_up]
    else:
        num_received_up += 1
        t = [spawn_meant[num_complete],teleport_meant[num_complete],assemble_meant[num_complete]]
        finish_station3(logger3,t,1)
        logger3.write_act("Pass",RNG.Expon(pass_meant[num_complete],3))
        num_complete += 1

### Station 4 - Body & ceiling

In [22]:
upstream_parts_idx = np.where(np.array(logger3.alog)=="Pass")[0]
upstream_parts_time = np.array(logger3.tlog)[upstream_parts_idx]

logger4 = ActionLogger()
num_received_up = 0
num_body = 0
num_complete = 0

# first body frame
t = [spawn_meant[num_body],teleport_meant[num_body],assemble_meant[num_body]]
make_body_frame(logger4,t,1)
num_body += 1

while True:
    if logger4.clock < upstream_parts_time[num_received_up]:
        # keep making body frames until getting parts from upstream
        t = [spawn_meant[num_body],teleport_meant[num_body],assemble_meant[num_body]]
        make_body_frame(logger4,t,1)
        num_body += 1
    else:
        num_received_up += 1
        t = [spawn_meant[num_complete],teleport_meant[num_complete],assemble_meant[num_complete]]
        finish_station4(logger4,t,1)
        logger4.write_act("Pass",RNG.Expon(pass_meant[num_complete],4))
        num_complete += 1

    # completed all necessary body frames
    if num_body == num_order:
        break

# empty out body frames in inventory
while num_received_up < num_order:
    if logger4.clock < upstream_parts_time[num_received_up]:
        # waiting parts coming from upstream
        logger4.clock = upstream_parts_time[num_received_up]
    else:
        num_received_up += 1
        t = [spawn_meant[num_complete],teleport_meant[num_complete],assemble_meant[num_complete]]
        finish_station4(logger4,t,1)
        logger4.write_act("Pass",RNG.Expon(pass_meant[num_complete],4))
        num_complete += 1

In [23]:
# player,action,time_stamp
log_df1 = pd.DataFrame({'player':"player1", 'action':logger1.alog, 'time_stamp':logger1.tlog})
log_df2 = pd.DataFrame({'player':"player2", 'action':logger2.alog, 'time_stamp':logger2.tlog})
log_df3 = pd.DataFrame({'player':"player3", 'action':logger3.alog, 'time_stamp':logger3.tlog})
log_df4 = pd.DataFrame({'player':"player4", 'action':logger4.alog, 'time_stamp':logger4.tlog})
log_df = pd.concat([log_df1,log_df2,log_df3,log_df4])
log_df = log_df.sort_values(by=['time_stamp'],ascending=True)
log_df.to_csv('./sim_data/assembly_line_4players.csv',index=False)

## Collaboration - Low

In [ ]:
logger1 = ActionLogger()
logger2 = ActionLogger()
logger3 = ActionLogger()
logger4 = ActionLogger()

num_complete = 0
while True:
    craft_car(logger1,2)
    logger1.write_act("Pass",RNG.Expon(pass_meant,2))
    craft_car(logger2,2)
    logger2.write_act("Pass",RNG.Expon(pass_meant,2))
    craft_car(logger3,2)
    logger3.write_act("Pass",RNG.Expon(pass_meant,2))
    craft_car(logger4,2)
    logger4.write_act("Pass",RNG.Expon(pass_meant,2))